In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import cv2
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/OpenPose/datasets/X_joint_data.csv')
df

,p_1,p_2
0,"[380.343, 134.948, 358.107, 162.386, 355.5, 16...","[495.219, 138.845, 517.383, 163.653, 514.745, ..."
1,"[380.305, 137.513, 358.076, 162.39, 352.898, 1...","[487.311, 137.525, 509.543, 162.376, 505.647, ..."
2,"[380.285, 134.96, 356.785, 162.383, 351.604, 1...","[484.767, 136.251, 506.958, 162.33, 505.55, 16..."
3,"[380.254, 134.954, 354.209, 162.368, 350.323, ...","[482.122, 134.921, 505.651, 162.403, 502.978, ..."
4,"[376.403, 138.798, 351.592, 162.372, 350.262, ...","[474.302, 137.56, 497.76, 163.622, 495.174, 16..."
...,...,...
7043,"[522.551, 144.064, 536.966, 181.901, 517.351, ...","[298.079, 161.022, 282.402, 193.69, 275.852, 1..."
7044,"[521.286, 144.074, 536.962, 181.909, 517.342, ...","[298.08, 161.009, 281.104, 193.665, 274.604, 1..."
7045,"[522.551, 144.066, 536.961, 181.906, 517.345, ...","[298.093, 161.012, 281.117, 193.689, 274.555, ..."
7046,"[522.55, 144.062, 536.961, 181.908, 517.343, 1...","[298.119, 161.01, 281.093, 193.652, 273.267, 1..."


In [5]:
df['p_1'] = df['p_1'].str.replace(pat='[', repl="", regex=True)
df['p_1'] = df['p_1'].str.replace(pat=']', repl="", regex=True)
df['p_2'] = df['p_2'].str.replace(pat='[', repl="", regex=True)
df['p_2'] = df['p_2'].str.replace(pat=']', repl="", regex=True)
df

,p_1,p_2
0,"380.343, 134.948, 358.107, 162.386, 355.5, 163...","495.219, 138.845, 517.383, 163.653, 514.745, 1..."
1,"380.305, 137.513, 358.076, 162.39, 352.898, 16...","487.311, 137.525, 509.543, 162.376, 505.647, 1..."
2,"380.285, 134.96, 356.785, 162.383, 351.604, 16...","484.767, 136.251, 506.958, 162.33, 505.55, 162..."
3,"380.254, 134.954, 354.209, 162.368, 350.323, 1...","482.122, 134.921, 505.651, 162.403, 502.978, 1..."
4,"376.403, 138.798, 351.592, 162.372, 350.262, 1...","474.302, 137.56, 497.76, 163.622, 495.174, 162..."
...,...,...
7043,"522.551, 144.064, 536.966, 181.901, 517.351, 1...","298.079, 161.022, 282.402, 193.69, 275.852, 19..."
7044,"521.286, 144.074, 536.962, 181.909, 517.342, 1...","298.08, 161.009, 281.104, 193.665, 274.604, 19..."
7045,"522.551, 144.066, 536.961, 181.906, 517.345, 1...","298.093, 161.012, 281.117, 193.689, 274.555, 1..."
7046,"522.55, 144.062, 536.961, 181.908, 517.343, 18...","298.119, 161.01, 281.093, 193.652, 273.267, 19..."


In [155]:
from google.colab.patches import cv2_imshow
import os


if not os.path.exists('Videos'):
    os.mkdir('Videos')

if not os.path.exists('Images'):
    os.mkdir('Images')


size = (480, 640)
fps = 30
img_arr = []


for i in range(49):
    img = np.zeros(size + (3,))

    p1 = df['p_1'][i].split(',')
    p2 = df['p_2'][i].split(',')

    for idx, xy in enumerate(p1):
        if (idx + 1)% 2 == 0 :
            x,y = (np.float32(p1[idx - 1]), np.float32(p1[idx]))
            cv2.circle(img, (x,y), 1, (0,255,0), cv2.LINE_4) # image, (x,y), radius, color)

    for idx, xy in enumerate(p2):
        if (idx + 1)% 2 == 0 :
            x,y = (np.float32(p2[idx - 1]), np.float32(p2[idx]))
            cv2.circle(img, (x,y), 1, (0,255,0), cv2.LINE_4) # image, (x,y), radius, color)

    img_path = './Images/img_' + str('%02d' %i) + '.jpg'
    cv2.imwrite(img_path, img)

In [158]:
!ffmpeg -f image2 -r 30 -i ./Images/img_%02d.jpg -vcodec libx264 ./test.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [160]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('test.mp4', width=640, height=480)

In [177]:
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/OpenPose/datasets/X_val.txt')


df2

,307.589,162.976,319.364,205.944,293.267,204.68,285.434,250.281,277.616,286.729,349.276,208.539,357.095,255.523,349.362,297.17,297.183,290.662,307.599,351.938,308.905,401.454,329.758,291.958,333.701,357.132,337.546,411.953,304.994,160.291,315.459,160.269,0,0.1,329.752,161.651
0,307.567,162.9790,319.362,205.947,293.257,204.695,285.428,250.280,277.616,285.504,349.282,208.541,357.101,255.503,349.361,297.149,297.182,290.671,307.603,351.914,307.689,401.429,329.757,291.969,333.698,357.127,337.545,411.937,304.969,160.2950,315.434,160.2680,0.000,0.000,328.527,161.6550
1,306.298,162.9510,319.351,205.937,291.991,204.679,285.432,251.551,277.617,286.773,349.279,208.536,357.113,255.539,349.360,297.175,297.188,290.672,307.602,350.672,308.904,401.455,329.767,291.976,333.709,357.113,337.550,411.952,302.419,160.2670,314.127,160.2600,0.000,0.000,328.495,161.6810
2,306.245,161.7040,319.334,204.683,291.987,204.640,285.429,250.285,276.408,286.803,348.059,207.273,357.122,255.515,349.363,297.180,297.178,289.395,307.605,350.637,308.909,401.456,329.758,291.963,334.988,355.864,337.565,411.939,301.123,159.0240,312.818,159.0280,0.000,0.000,328.481,161.6570
3,305.000,161.6520,318.097,204.647,290.694,203.389,285.438,250.276,276.403,286.831,348.016,207.238,357.130,255.473,349.387,297.168,297.178,289.377,307.625,350.636,308.907,401.456,329.769,290.673,334.992,355.857,337.551,411.944,301.089,157.7380,311.516,158.9770,0.000,0.000,328.431,161.6530
4,303.722,161.6330,318.080,203.405,291.953,203.334,285.440,250.270,276.373,288.051,347.981,207.227,357.154,255.548,349.406,298.497,297.200,288.108,307.627,349.391,308.915,401.437,329.777,290.681,334.985,357.096,337.558,411.943,299.783,157.7150,308.970,157.7260,0.000,0.000,327.182,161.6010
5,302.412,161.6700,318.023,203.412,288.099,203.394,285.411,251.558,276.385,288.083,346.733,204.690,358.420,255.483,350.666,297.139,297.222,288.099,307.608,350.613,308.914,401.449,331.092,289.391,333.690,357.095,337.559,411.958,298.474,158.9800,308.908,158.9800,0.000,0.000,327.123,161.6270
6,299.781,161.6780,316.793,203.395,288.043,203.394,285.392,251.555,277.618,286.814,346.684,204.638,358.423,254.105,354.528,293.274,297.263,289.370,307.602,350.632,308.919,401.443,332.404,289.395,333.670,355.863,337.579,411.910,297.190,158.9700,307.611,158.9990,0.000,0.000,325.838,161.6510
7,297.190,162.9550,316.709,204.640,286.754,204.683,285.401,251.534,278.943,286.723,344.119,204.639,358.458,252.861,357.098,294.561,298.482,289.377,306.367,350.668,308.915,401.438,333.691,288.102,332.363,355.850,337.604,411.882,295.841,160.2770,306.274,160.2450,0.000,0.000,321.942,161.6600
8,295.942,162.9700,315.458,207.249,285.506,209.849,285.388,254.103,277.676,288.050,342.822,205.942,358.491,254.148,358.441,293.282,299.809,291.972,306.320,354.542,308.922,401.508,336.240,291.959,329.762,357.146,336.317,411.932,293.291,160.3150,305.015,160.2600,0.000,0.000,318.103,162.9640
9,293.291,170.7610,312.854,211.148,285.390,212.436,285.426,254.146,284.161,285.468,340.238,209.825,359.753,255.538,358.503,297.222,302.417,293.262,306.268,357.123,308.944,402.764,337.550,291.975,328.463,358.413,336.247,413.173,288.100,164.2280,298.527,162.9490,0.000,0.000,317.996,168.1210
